# AZ Vibration Review 

In [ ]:
import math
import numpy as np

from nptdms import tdms

import matplotlib.pyplot as plt

from acoustics import Signal
from acoustics.standards.iec_61672_1_2013 import (
    NOMINAL_OCTAVE_CENTER_FREQUENCIES,
    NOMINAL_THIRD_OCTAVE_CENTER_FREQUENCIES)

import scipy.fftpack
from scipy import integrate
import scipy.signal as signal

import scripts.sig as sig
from scripts.vib_files import getTdmsFilesInPath

%matplotlib inline

### Path to TDMS file and Appropriate Limits:

In [ ]:
#Find all vibration files in this path
mypath = './TDMS/'
result = getTdmsFilesInPath(mypath)
print ('{0} files found\n'.format(len(result)))

In [ ]:
[print(filename) for filename in result]
print()

In [ ]:
#fn = <META_FILENAME>  #uncomment to get a scriptable choice
fn = result[2]
print(fn)

In [ ]:
tdmspath = fn
#print(getlimitfrompath(fn))

#lim = getlimitfrompath(fn)
tdfile = tdms.TdmsFile(tdmspath)

#df = tdfile.as_dataframe()
#df.plot(figsize=(15,24), color=('r'), subplots=True );

In [ ]:
tdfile.object().properties

In [ ]:
for g in tdfile.groups():
    print(g)
    print(tdfile.object(g).properties)
    print('******************************************')
    for c in tdfile.group_channels(g):
        print(c.properties)
    print('------------------------------------------')

### Plot overall Run

In [ ]:
win = np.hanning(8192)

class Axis():
    pass

axis = Axis()
axis.Name = []
axis.f = []
axis.Pxx = []
axis.dBPxx = []

for channel in tdfile.group_channels('Vibration'):
    #Get Metadata from tdms channel
    chName = (channel.properties['NI_ChannelName'])
    Sensitivity = float(channel.properties['Sensor Sensitivity (mV/EU)'])
    data = (channel.data * 1000 / Sensitivity)
    axis.Name.append(chName)
   
    #generate PSD using Welch's method
    f, Pxx_spec = signal.welch(data, fs=8533, window='hann', nfft=8192, detrend=None, scaling='spectrum')
    axis.f.append(f)
    axis.Pxx.append(Pxx_spec)
    axis.dBPxx.append(10*np.log10(Pxx_spec))
    

#PLOT LINEAR PSD    
fig, ax = plt.subplots()
fig.set_size_inches(20,10)

for i in range(0, 3):
    ax.plot(axis.f[i], axis.Pxx[i], label=axis.Name[i])
    
    #Area under g^2 / Hz Pxx curve is g^2 RMS, f[i] is frequency in Hz
    #Using trapezoid rule to numerically integrate area under curve, square root, and round to 2 places for g RMS data
    print(f'{axis.Name[i]}: {round(math.sqrt(integrate.trapz(axis.Pxx[i], x=axis.f[i])),2)} g RMS; 4kHz bandpass')

plt.xlabel('frequency [Hz]')
plt.ylabel('Linear spectrum [g^2/Hz]')
#plt.xlim(xmin=16)
plt.legend()
plt.show()


#PLOT LOG(x) PSD
fig, ax = plt.subplots()
fig.set_size_inches(20,10)

for i in range(0, 3):
    ax.semilogx(axis.f[i], axis.Pxx[i], label=axis.Name[i])
plt.xlabel('log 10 (frequency [Hz])')
plt.ylabel('Linear spectrum [g^2/Hz]')
plt.legend()
plt.show()

In [ ]:
for channel in tdfile.group_channels('Vibration'):
    chName = (channel.properties['NI_ChannelName'])
    DOCSensitivity = float(channel.properties['Sensor Sensitivity (mV/EU)'])
    data = (channel.data * 1000 / DOCSensitivity)
    
    fig, ax = plt.subplots()
    fig.set_size_inches(20,10)
    
    ax.set_title(chName + ' Spectrogram')
    ax.set_xlabel('time (seconds)')
    ax.set_ylabel('frequency (Hz)')
    
    NFFT = 8192
    cmap = plt.get_cmap('magma')
    vmin = 10*np.log10(np.max(data)) - 80 #clamp to -80 dB
    cmap.set_under(color='k', alpha=None)
    
    pxx, freq, t, cax = plt.specgram(data, NFFT=NFFT, Fs=8533.3333, Fc=None, 
                                     detrend=None, window=np.hanning(8192), mode='psd',  
                                     noverlap=NFFT*0.75, pad_to=None, cmap=cmap, vmin=vmin, scale='dB')
    fig.colorbar(cax).set_label('Intensity [dB(g^2/Hz)]')
    
    plt.show()
    rmsData = math.sqrt(sum(data*data)/len(data))